In [76]:
import psycopg2
import pandas as pd

# Credenciales de la base de datos
dbname = "DBInmerso"
user = "postgres"
password = "postgres"
host = "localhost"  # Cambia esto según tu configuración
port = "5432"

# Crear la cadena de conexión
connection_string = f"dbname={dbname} user={user} password={password} host={host} port={port}"

# Intentar establecer la conexión
try:
    connection = psycopg2.connect(connection_string)

    # Consulta para concatenar las tablas
    query_concatenar = """
        SELECT
            u.dni,
            u.nombre,
            u.apellido,
            u.edad,
            u.fecha_de_nacimiento,
            u.id_solicitud,
            u.usuario_solicitante,
            u.oficio_especial,
            r.ingresos,
            d.grado_dis,
            fn.tipo_fam_num,
            p.valoracion_patrimonio,
            p.coche,
            p.arrendador
        FROM
            usuarios u
        JOIN renta r ON u.dni = r.dni
        JOIN discapacidad d ON u.dni = d.dni
        JOIN familia_numerosa fn ON u.dni = fn.dni
        JOIN patrimonio p ON u.dni = p.dni;
    """

    # Crear un DataFrame con los resultados de la consulta concatenada
    df = pd.read_sql_query(query_concatenar, connection)

except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

finally:
    # Cerrar la conexión en el bloque 'finally' para asegurar que se cierre
    if connection:
        connection.close()

df

C:\Users\DELL\AppData\Local\Temp\ipykernel_13356\1002600950.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_concatenar, connection)


,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador
0,95822412,Antonio,Montoya,73,1945-08-12,52,False,True,4154.0,0,2,2000000.0,True,False
1,46913810,Salud,Bermejo,77,1918-01-13,458,True,True,739.0,2,0,120426.0,False,True
2,28728463,Cándido,Armas,73,1934-11-05,1386,False,False,883.0,0,0,167774.0,False,True
3,83197857,Victor,Mendizábal,72,1941-12-25,1210,True,True,4168.0,2,1,110058.0,True,False
4,66629388,Pelayo,Jaume,71,1952-04-11,62,False,True,1893.0,0,1,492729.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,55047826,Ciriaco,Puerta,90,1929-07-17,183,False,True,301.0,1,2,96328.0,True,False
1496,39830361,Juan Carlos,Portero,81,1935-07-08,458,False,True,583.0,0,0,423781.0,True,True
1497,51216287,Pedro,Estévez,83,1931-09-14,679,False,False,2630.0,2,2,25215.0,True,True
1498,59886371,Tecla,Calvo,78,1909-04-06,1200,False,True,433.0,1,2,396242.0,False,False


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dni                  1500 non-null   int64 
 1   nombre               1500 non-null   object
 2   apellido             1500 non-null   object
 3   edad                 1500 non-null   int64 
 4   fecha_de_nacimiento  1500 non-null   object
 5   id_solicitud         1500 non-null   int64 
 6   usuario_solicitante  1500 non-null   bool  
 7   oficio_especial      1500 non-null   bool  
dtypes: bool(2), int64(3), object(3)
memory usage: 73.4+ KB


### EDAD.SCORE

In [77]:
# Definir la función que asigna el puntaje según las condiciones dadas
def asignar_puntaje_edad_nueva(edad):
    if edad < 60:
        return 1
    elif 60 <= edad < 70:
        return 20
    elif 70 <= edad < 80:
        return 30
    else:  # Edad >= 80
        return 40

df_edad = df

# Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
df_edad['score_edad'] = df_edad['edad'].apply(asignar_puntaje_edad_nueva)

# Calcular el Min-Max Scaling para la columna 'score_edad'
min_valor = df_edad['score_edad'].min()
max_valor = df_edad['score_edad'].max()
rango_deseado = 100
min_deseado = 0

df_edad['score_edad_scaled'] = ((df_edad['score_edad'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

# Imprimir el DataFrame con las columnas 'dni', 'edad', 'score_edad_scaled' ordenado por 'edad'
df_edad = df_edad[['dni', 'edad','score_edad', 'score_edad_scaled']].sort_values(by='dni')
df_edad





,dni,edad,score_edad,score_edad_scaled
178,10054484,79,30,0.0
344,10154493,87,40,100.0
644,10209961,89,40,100.0
1352,10241310,88,40,100.0
1153,10331931,72,30,0.0
...,...,...,...,...
222,99774697,82,40,100.0
149,99913412,77,30,0.0
498,99927658,73,30,0.0
1360,99929175,90,40,100.0
